In [1]:
import pandas as pd
import numpy as np
import utils

In [2]:
df = pd.read_json("data_no_bool.json").T

In [3]:
df.head()

,label,time_series
0,6,"[{'BallAcceleration': 1636.7987723122642, 'Dis..."
1,-1,"[{'BallAcceleration': 0.0, 'DistanceWall': 39...."
2,-1,"[{'BallAcceleration': 124246.29375405672, 'Dis..."
3,-1,"[{'BallAcceleration': -8210.634011562943, 'Dis..."
4,-1,"[{'BallAcceleration': 1197.5360615055833, 'Dis..."


In [4]:
df.time_series = df.time_series.apply(utils.extract_matrix)
df['shape'] = df.time_series.apply(lambda x: x.shape)
df.head()

,label,time_series,shape
0,6,"[[1636.7987723122642, 3498.01, 2012.98, 299.66...","(32, 17)"
1,-1,"[[0.0, 39.7800000000002, 1704.5, 193.421147499...","(39, 17)"
2,-1,"[[124246.29375405672, 35.48999999999978, 1450....","(34, 17)"
3,-1,"[[-8210.634011562943, 36.0, 1036.28, 579.43975...","(23, 17)"
4,-1,"[[1197.5360615055833, 3975.02, 2013.04, 142.93...","(22, 17)"


In [5]:
S, v_t_list = utils.compute_S_matrix(df.time_series)

In [6]:
w = utils.compute_weight_vector(S, aggregation='mean', algorithm='1')
w

array([7.33901974e-01, 2.11399585e-01, 5.46184777e-02, 6.73521064e-05,
       1.01007818e-05, 2.51086069e-06, 2.46274980e-11, 1.48000471e-11,
       9.75667933e-12, 6.57018216e-12, 4.07295441e-12, 2.01973052e-12,
       8.18530946e-13, 2.46108301e-13, 7.03280356e-14, 2.61036403e-14,
       5.54693561e-15])

Eros norm:
$$
EROS(\mathbf{A,B},w) = \sum_{i=1}^n w_i\cdot|<a_i,b_i>|
$$

In [7]:
eros = utils.eros_norm(w, A=v_t_list[0], B=v_t_list[0])
eros

0.9999999999999998

In [8]:
K_eros = utils.compute_kernel_matrix(len(df), weight_vector=w, v_t_list=v_t_list)

In [9]:
K_eros[0,1] == K_eros[1,0]

True

In [10]:
eig_vecs = utils.perform_PCA(len(df), weight_vector=w, v_t_list=v_t_list)

In [11]:
df = pd.read_json("data_no_bool.json").T
df.head()

#from sklearn.utils import shuffle
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
shuffled_df.head()

from utils import extract_matrix
# Create the matrix version
shuffled_df.time_series = shuffled_df.time_series.apply(extract_matrix)
shuffled_df["shape"] = shuffled_df.time_series.apply(lambda x: x.shape)
shuffled_df.head()

X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], train_size=.75, random_state=seed)

In [12]:
S, v_t_list_train = utils.compute_S_matrix(X_train)

In [13]:
_, v_t_list_test = utils.compute_S_matrix(X_test)

In [14]:
K_eros_train = utils.compute_kernel_matrix(len(X_train), weight_vector=w, v_t_list=v_t_list_train)
V = utils.perform_PCA(len(X_train), weight_vector=w, v_t_list=v_t_list_train)

K_eros_test = utils.compute_test_kernel_matrix(len(X_train), len(X_test), w, v_t_list_train, v_t_list_test)

In [15]:
N_test = len(X_test)
N_train = len(X_train)
O_test = np.ones(shape=(N_test, N_train))*(1/N_train)
O_train = np.ones(shape=(N_train, N_train))*(1/N_train)

K_eros_test_mc = K_eros_test - O_test@K_eros_train - K_eros_test@O_train + O_test@K_eros_train@O_train

Y = K_eros_test_mc @ V

the $i$-th MTS item in the test set is represented as features (length = num examples in the training set) in the $i$-th row of $\mathbf{Y}$

In [16]:
Y.shape

(75, 223)

In [17]:
Y

array([[ 1.24680009e+00, -1.27473153e+00,  1.60465999e+00, ...,
        -3.92937611e-03,  3.92455922e-03,  1.26002191e-03],
       [-2.72291846e+00, -5.85372969e-01, -2.98740898e-01, ...,
        -3.19158956e-04, -8.21647680e-04, -4.86970189e-04],
       [-2.70571600e-01,  2.99148236e-01, -1.72580986e+00, ...,
         3.57826704e-04,  2.98196470e-03, -4.27097760e-03],
       ...,
       [ 1.80098641e+00, -1.55540764e+00, -2.17269400e+00, ...,
         1.33004254e-03,  5.35847323e-04, -3.31968017e-03],
       [-1.27804529e+00,  3.92497578e-01, -1.18415776e+00, ...,
         1.04530652e-03, -4.13398668e-03,  1.61742561e-03],
       [ 1.64495238e+00, -5.22117689e-01, -2.54705714e+00, ...,
         2.48748935e-03, -1.86290432e-02,  9.13825104e-03]])